In [37]:
dice(x) = (x[1] - 1)^2 + sum(i*(2 * x[i] ^ 2 - x[i - 1])^2 for i in 2:length(x))

dice (generic function with 1 method)

In [38]:
# x = 0:0.1:10
# y = 0:0.1:10

# # Create a grid of (x, y) values
# X = [x_val for x_val in x, y_val in y]
# Y = [y_val for x_val in x, y_val in y]

# # Calculate the dice values for the grid
# Z = [dice([x_val, y_val]) for x_val in x, y_val in y]
# using Plots

# plot(
#     x, y, Z, st=:surface, legend=false, c=:viridis,
#     xlabel="X", ylabel="Y", zlabel="Dice(x, 2)"
# )

In [39]:
no_of_gen = 20
dimension = 3
lower_bound = -10.0
upper_bound = 10.0
chromosome = rand(Float64, no_of_gen, dimension) .* (upper_bound - lower_bound) .+ lower_bound

20×3 Matrix{Float64}:
 -4.6826     8.11075  -6.99974
  9.3957    -6.58418   6.75901
  1.05946   -0.12535  -8.85198
  9.59188   -5.03735  -9.9064
  0.872807   2.37262   8.25195
 -4.23067    1.337     8.50673
 -9.19641   -1.76239   0.135268
  6.77242    2.19326  -8.782
  7.06669   -7.89904  -9.07935
 -6.57521    6.14384  -7.70558
 -5.47373   -5.25739   9.77193
 -2.41677   -7.78275   2.21229
  0.046626  -9.18075   7.87244
 -6.79002    5.10576   1.71728
 -8.7496     1.46197   2.39585
  4.31109   -2.28857  -2.76363
 -8.45416   -6.26507   6.60888
  6.04773    9.1521   -0.0347965
  5.53755   -1.98976   9.20591
  7.88447   -8.13715   8.49899

In [40]:
func_values = dice.(eachrow(chromosome))
println(sum(func_values) / length(func_values))

60597.78367103997


In [41]:
fitness = 1 ./(func_values .+ 1) # In order to avoid dividing them by zero

20-element Vector{Float64}:
 1.6287103121556946e-5
 2.450472388451297e-5
 1.3550127405385157e-5
 7.997372574551822e-6
 1.8539943543430683e-5
 1.6172322510108074e-5
 0.0016963007464223486
 1.440679030583468e-5
 8.525004861634204e-6
 1.9390885944498853e-5
 8.133144296469923e-6
 3.177354251222032e-5
 9.092916333087078e-6
 0.00014268405937602308
 0.0013579315126989325
 0.0009867499455024123
 2.4092278493934686e-5
 1.907472686543858e-5
 1.1330575911495701e-5
 9.907594516239344e-6

In [42]:
indi_prob = fitness ./ sum(fitness)
cumul_prob = cumsum(indi_prob)

20-element Vector{Float64}:
 0.00367120565170776
 0.00919470974859607
 0.012248985511495204
 0.014051638762683106
 0.018230647455082117
 0.021875980904329278
 0.4042318142765264
 0.4074791867281532
 0.409400771296955
 0.4137715872449945
 0.4156048442142978
 0.42276678136021356
 0.4248163764038129
 0.4569781746855805
 0.7630636588165258
 0.985482707171178
 0.9909132437318746
 0.9952127956258203
 0.997766771861676
 1.0

In [43]:
roulette = rand(length(cumul_prob))

20-element Vector{Float64}:
 0.7347618388900107
 0.6478004946716386
 0.0011190649921971607
 0.5192663086450545
 0.22150068977020976
 0.5170482666052582
 0.5926570708153803
 0.11212341593768893
 0.8234676608172142
 0.6639152761233239
 0.527834839880947
 0.8542624527643009
 0.8889186054375696
 0.3614463542708377
 0.9089823353572997
 0.5119707777036939
 0.07574660797434718
 0.7767430480090828
 0.6132730114090549
 0.9068639968403536

In [44]:
new_chromo = []
for i = 1:length(roulette)
    for j = 1:length(roulette)-1
        if roulette[i] > cumul_prob[j] && roulette[i] <= cumul_prob[j+1]
            push!(new_chromo, chromosome[j+1, :])
        end
    end
end
new_chromo

19-element Vector{Any}:
 [-8.74960130457183, 1.4619677549951486, 2.395850036750282]
 [-8.74960130457183, 1.4619677549951486, 2.395850036750282]
 [-8.74960130457183, 1.4619677549951486, 2.395850036750282]
 [-9.196408574509611, -1.7623943182380906, 0.13526838814211573]
 [-8.74960130457183, 1.4619677549951486, 2.395850036750282]
 [-8.74960130457183, 1.4619677549951486, 2.395850036750282]
 [-9.196408574509611, -1.7623943182380906, 0.13526838814211573]
 [4.311092597353294, -2.288570195435198, -2.7636349101905306]
 [-8.74960130457183, 1.4619677549951486, 2.395850036750282]
 [-8.74960130457183, 1.4619677549951486, 2.395850036750282]
 [4.311092597353294, -2.288570195435198, -2.7636349101905306]
 [4.311092597353294, -2.288570195435198, -2.7636349101905306]
 [-9.196408574509611, -1.7623943182380906, 0.13526838814211573]
 [4.311092597353294, -2.288570195435198, -2.7636349101905306]
 [-8.74960130457183, 1.4619677549951486, 2.395850036750282]
 [-9.196408574509611, -1.7623943182380906, 0.13526838814

In [45]:
length(new_chromo)

19

In [46]:
while length(new_chromo)!=no_of_gen
    push!(new_chromo, chromosome[end, :])
end
chromosome = mapreduce(permutedims, vcat, new_chromo)

20×3 Matrix{Float64}:
 -8.7496    1.46197   2.39585
 -8.7496    1.46197   2.39585
 -8.7496    1.46197   2.39585
 -9.19641  -1.76239   0.135268
 -8.7496    1.46197   2.39585
 -8.7496    1.46197   2.39585
 -9.19641  -1.76239   0.135268
  4.31109  -2.28857  -2.76363
 -8.7496    1.46197   2.39585
 -8.7496    1.46197   2.39585
  4.31109  -2.28857  -2.76363
  4.31109  -2.28857  -2.76363
 -9.19641  -1.76239   0.135268
  4.31109  -2.28857  -2.76363
 -8.7496    1.46197   2.39585
 -9.19641  -1.76239   0.135268
  4.31109  -2.28857  -2.76363
 -8.7496    1.46197   2.39585
  4.31109  -2.28857  -2.76363
  7.88447  -8.13715   8.49899

In [47]:
#Crossover
crossover_rate = 0.3
parent_index = []
for i in 1:length(new_chromo)
    crossover_rand = rand()
    if crossover_rand <= crossover_rate
        push!(parent_index, i) # Selecting parents
    end
end
parent_index

2-element Vector{Any}:
  7
 19

In [48]:
child = []
if length(parent_index)>0
    for i in 1:length(parent_index)
        first_parent = chromosome[parent_index[i], :]
        if length(parent_index)>1
            second_parent = chromosome[rand((parent_index[parent_index.!=parent_index[i]])), :]
        else
            second_parent = first_parent
        end
        crossover_point = rand(1:length(chromosome[1, :])-1)
        push!(child, [first_parent[1:crossover_point]; second_parent[crossover_point+1:end]])
    end
    for i in 1:length(parent_index)
        chromosome[i, :] = child[i] # Replacing parents with children
    end
end
chromosome

20×3 Matrix{Float64}:
 -9.19641  -1.76239  -2.76363
  4.31109  -1.76239   0.135268
 -8.7496    1.46197   2.39585
 -9.19641  -1.76239   0.135268
 -8.7496    1.46197   2.39585
 -8.7496    1.46197   2.39585
 -9.19641  -1.76239   0.135268
  4.31109  -2.28857  -2.76363
 -8.7496    1.46197   2.39585
 -8.7496    1.46197   2.39585
  4.31109  -2.28857  -2.76363
  4.31109  -2.28857  -2.76363
 -9.19641  -1.76239   0.135268
  4.31109  -2.28857  -2.76363
 -8.7496    1.46197   2.39585
 -9.19641  -1.76239   0.135268
  4.31109  -2.28857  -2.76363
 -8.7496    1.46197   2.39585
  4.31109  -2.28857  -2.76363
  7.88447  -8.13715   8.49899

In [49]:
# Mutation
mutation_rate = 0.06 # 6%
mutation_indices = rand(1:length(chromosome), round(Int, mutation_rate * length(chromosome)))

4-element Vector{Int64}:
 11
 40
 17
 25

In [50]:
for index in unique(mutation_indices)
    chromosome[index] = rand(Float64) .* (upper_bound - lower_bound) .+ lower_bound
end
chromosome

20×3 Matrix{Float64}:
 -9.19641  -1.76239  -2.76363
  4.31109  -1.76239   0.135268
 -8.7496    1.46197   2.39585
 -9.19641  -1.76239   0.135268
 -8.7496    1.51762   2.39585
 -8.7496    1.46197   2.39585
 -9.19641  -1.76239   0.135268
  4.31109  -2.28857  -2.76363
 -8.7496    1.46197   2.39585
 -8.7496    1.46197   2.39585
  8.68785  -2.28857  -2.76363
  4.31109  -2.28857  -2.76363
 -9.19641  -1.76239   0.135268
  4.31109  -2.28857  -2.76363
 -8.7496    1.46197   2.39585
 -9.19641  -1.76239   0.135268
 -6.47441  -2.28857  -2.76363
 -8.7496    1.46197   2.39585
  4.31109  -2.28857  -2.76363
  7.88447  -6.03187   8.49899

In [51]:
minimum(dice.(eachrow(chromosome)))

27.899833586558128

In [52]:
func_values = dice.(eachrow(chromosome))
println(sum(func_values) / length(func_values))

4600.2891088585
